# Simulate observed volumes for distance | covariates
from fit model of Observed Vol ~ dist

In [1]:
from opensoundscape import Audio, Spectrogram, CNN, BoxedAnnotations

import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for big visuals
%config InlineBackend.figure_format = 'retina'


/Users/SML161/opensoundscape/opensoundscape/ml/cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
clips = pd.read_csv('../metadata/localized_clips_with_decibel.csv')

In [3]:
clips[clips.class_name=='Catharus guttatus']

,receiver_id,event_id,clip_file,distance_m,event_position_x,event_position_y,event_position_z,receiver_position_x,receiver_position_y,receiver_position_z,...,class_name,json_idx,include,arrival_angle_deg,corrected_dB_max,habitat_rugosity_tri,habitat_tree_cover,approx_recording_time,rel_hum,temp_C
0,MSD-1636,grid3_Catharus_guttatus_grid3_23,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,230.302210,-35.276684,-97.523188,47.179542,32.485285,117.796432,1.51900,...,Catharus guttatus,23.0,True,162.530997,51.276161,0.927883,0.611341,2023-06-03 11:45:36.567955+00:00,96.0,12.3
1,MSD-1636,grid3_Catharus_guttatus_grid3_49,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,163.881318,-18.446444,-35.343809,29.999030,32.485285,117.796432,1.51900,...,Catharus guttatus,49.0,True,161.603789,45.259764,0.927883,0.611341,2023-06-03 11:48:57.927830+00:00,96.0,12.3
2,MSD-1636,grid3_Catharus_guttatus_grid3_54,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,119.349469,-7.087313,6.751877,20.157993,32.485285,117.796432,1.51900,...,Catharus guttatus,54.0,True,160.385618,51.390041,0.927883,0.611341,2023-06-03 11:49:13.427913+00:00,96.0,12.3
3,MSD-1636,grid3_Catharus_guttatus_grid3_60,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,175.145571,-25.365534,-46.015651,23.764344,32.485285,117.796432,1.51900,...,Catharus guttatus,60.0,True,160.549146,50.844738,0.927883,0.611341,2023-06-03 11:50:42.000246+00:00,96.0,12.3
4,MSD-1636,grid3_Catharus_guttatus_grid3_63,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,135.489937,-8.837048,-8.562455,27.661294,32.485285,117.796432,1.51900,...,Catharus guttatus,63.0,True,161.891036,51.094807,0.927883,0.611341,2023-06-03 11:50:58.427205+00:00,96.0,12.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,MSD-3932,grid3_Catharus_guttatus_grid3_49,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,188.479573,-18.446444,-35.343809,29.999030,16.601533,148.031190,4.11856,...,Catharus guttatus,49.0,True,169.179704,48.206388,0.929358,0.543173,2023-06-03 11:48:58+00:00,96.0,12.3
82,MSD-3932,grid3_Catharus_guttatus_grid3_54,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,144.146693,-7.087313,6.751877,20.157993,16.601533,148.031190,4.11856,...,Catharus guttatus,54.0,True,170.481539,48.221415,0.929358,0.543173,2023-06-03 11:49:13.500000+00:00,96.0,12.3
83,MSD-3932,grid3_Catharus_guttatus_grid3_60,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,199.502802,-25.365534,-46.015651,23.764344,16.601533,148.031190,4.11856,...,Catharus guttatus,60.0,True,167.796430,46.288027,0.929358,0.543173,2023-06-03 11:50:42.072996+00:00,96.0,12.3
84,MSD-3932,grid3_Catharus_guttatus_grid3_64,/bgfs/jkitzes/sml161/data/loca2023_grid3_resam...,177.914223,-16.591039,-23.331029,38.567272,16.601533,148.031190,4.11856,...,Catharus guttatus,64.0,True,169.037658,48.925103,0.929358,0.543173,2023-06-03 11:50:58.500000+00:00,96.0,12.3


In [ ]:
# estimate L_obs
# known inputs: distance, hab, sigma (of linear model Y = Norm(alpha_hab,sigma) )
#
# Y =  -1*(L_obs + A_geo + A_atm)/(freq*d)
# and Y is modeled as 
# Y = Norm(alpha_hab, l_obs)
# alpha_hab = a0 + a1*habitat_tree_cover + fixed_effect_class
# then L_obs = -1*(Y*freq*d)-A_geo-A_atm
# 

# fit-model parameters:
intercept,habitat_tree_cover,class_nameCatharus_guttatus = -0.0002688748,-0.0012270960, 0.0007018519

